In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble._forest import _generate_unsampled_indices
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [5]:
def generate_data(nu, sigma, n_observations=5000, n_predictors=30, p_success=0.5, test_size=0.9):
    # Generate the predictor variables (X1, X2, ..., X30)
    predictors = np.random.binomial(1, p_success, size=(n_observations, n_predictors))

    # Select the first ν predictors to determine the response variable
    selected_predictors = predictors[:, :nu]

    # Generate the random error term ε from a normal distribution
    epsilon = np.random.normal(0, sigma, size=n_observations)

    # Calculate the response variable y based on the provided formula
    y = (np.mean(selected_predictors, axis=1) + epsilon > 0.5).astype(int)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, y, test_size=test_size)

    return X_train, X_test, y_train, y_test
    
def train_with_oob_samples(X_train, y_train, n_estimators=200):
    # Initialize lists and arrays to store results
    tree_regressors = []
    oob_indices_per_tree = []
    
    X_full_train = np.empty((0, X_train.shape[1]))
    y_full_train = np.array([])
    y_full_pred = np.array([])

    # Fit each decision tree regressor to a bootstrapped sample of the data
    for _ in range(n_estimators):
        # Create a bootstrapped sample of the data
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_train_bootstrap = X_train[indices]
        y_train_bootstrap = y_train[indices]

        # Train the decision tree regressor on the bootstrap sample
        tree_regressor = DecisionTreeRegressor()

        tree_regressor.fit(X_train_bootstrap, y_train_bootstrap)
        tree_regressors.append(tree_regressor)

        # Identify out-of-bag (OOB) indices
        oob_indices = [i for i in range(len(X_train)) if i not in indices]
        X_oob = X_train[oob_indices]
        y_oob = y_train[oob_indices]
        
        # Predict the OOB samples
        y_oob_pred = tree_regressor.predict(X_oob)

        # Store OOB indices for each tree
        oob_indices_per_tree.append(oob_indices)

        # Concatenate OOB samples and predictions
        X_full_train = np.concatenate((X_full_train, X_oob), axis=0)
        y_full_train = np.concatenate((y_full_train, y_oob), axis=0)
        y_full_pred = np.concatenate((y_full_pred, y_oob_pred), axis=0)
    
    return X_full_train, y_full_pred

def get_max_depth_and_accuracy(X_train, y_train, X_test, y_test):
    
    # Create and fit a Decision Tree Classifier with no maximum depth
    dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=None)
    dt_classifier.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = dt_classifier.predict(X_test)

    # Calculate test accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Retrieve the maximum depth of the fitted tree
    max_depth = dt_classifier.tree_.max_depth

    return max_depth, accuracy
    
def evaluate_tree_depth(X_train, y_train, X_test, y_test, X_full_train, y_full_pred, max_depth):
    
    # Initialize lists to store accuracies
    train_accuracy_list = []
    imputed_accuracy_list = []
    test_accuracy_list = []

    best_imputed_accuracy = 0
    best_depth_for_imputed = 0
    best_test_accuracy = 0
    oracle_test_accuracy = 0
    oracle_depth_for_test = 0

    # Iterate over depths from max_depth down to 1
    for this_depth in range(max_depth, 0, -1):
        
        # Initialize and fit the Decision Tree Classifier
        dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=this_depth)
        dt_classifier.fit(X_train, y_train)

        # Imputed accuracy on X_full_train and y_full_pred
        y_pred_imputed = dt_classifier.predict(X_full_train)
        imputed_accuracy = accuracy_score(y_full_pred, y_pred_imputed)
        imputed_accuracy_list.append(imputed_accuracy)

        # Training accuracy
        y_pred_train = dt_classifier.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_pred_train)
        train_accuracy_list.append(train_accuracy)

        # Test accuracy
        y_pred_test = dt_classifier.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred_test)
        test_accuracy_list.append(test_accuracy)

        # Track the best imputed accuracy and corresponding depth
        if imputed_accuracy > best_imputed_accuracy:
            best_imputed_accuracy = imputed_accuracy
            best_depth_for_imputed = this_depth
            best_test_accuracy = test_accuracy

        # Track the best test accuracy and corresponding "oracle" depth
        if test_accuracy > oracle_test_accuracy:
            # best_test_accuracy = test_accuracy
            oracle_test_accuracy = test_accuracy
            oracle_depth_for_test = this_depth

        # Print the current depth and accuracies
        # print(f"Training Accuracy: {train_accuracy:.3f}")
        # print(f"Imputed Accuracy: {imputed_accuracy:.3f}")
        # print(f"Test Accuracy: {test_accuracy:.3f}")

    return best_depth_for_imputed, best_test_accuracy, oracle_depth_for_test, oracle_test_accuracy

def CV_find_best_depth_and_accuracy(X_train, y_train, X_test, y_test, max_depth):
    # Define parameter grid based on the given max_depth
    parameter = {'max_depth': list(range(1, max_depth + 1))}
    
    # Initialize the Decision Tree model
    model = DecisionTreeClassifier()
    
    # Perform Grid Search with cross-validation
    cv = GridSearchCV(model, param_grid=parameter, cv=5)
    cv.fit(X_train, y_train)
    
    # Get the best depth from the Grid Search
    best_depth = cv.best_params_['max_depth']
    # print("Best max_depth:", best_depth)

    # Predict using the best model from the cross-validation
    y_pred = cv.predict(X_test)

    # Calculate the accuracy on the test set
    test_accuracy = accuracy_score(y_test, y_pred)

    return best_depth, test_accuracy

def evaluate_imputation(X_test, y_test, X_full_train, y_full_pred):
    dt_classifier = DecisionTreeClassifier(criterion='gini')
    dt_classifier.fit(X_full_train, y_full_pred)
    y_pred_test = dt_classifier.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    
    return(test_accuracy)

In [25]:
def evaluate_multiple_runs(N, nu, sigma, bootstrap_samples=1000):
    # Initialize lists to store accuracies and depths
    np.random.seed(123)
    AM_test_accuracies = []
    CV_test_accuracies = []
    
    AM_depths = []
    CV_depths = []
    
    oracle_depths = []
    oracle_test_accuracies = []
    
    # Variables to count frequency of consistency with oracle depth
    AM_consistent_with_oracle = 0
    CV_consistent_with_oracle = 0
    
    # Run the process N times
    for i in range(N):
        
        # Generate data and train with OOB samples
        X_train, X_test, y_train, y_test = generate_data(nu, sigma, n_observations = 1000, n_predictors= 50)

        X_full_train, y_full_pred = train_with_oob_samples(X_train, y_train)
        
        # Get the maximum possible depth of the tree
        max_depth, _ = get_max_depth_and_accuracy(X_train, y_train, X_test, y_test)
        
        # Evaluate the AM method
        depth_AM, AM_test_accuracy, oracle_depth_for_test, oracle_test_accuracy = evaluate_tree_depth(
            X_train, y_train, X_test, y_test, X_full_train, y_full_pred, max_depth
        )
        
        
        # Evaluate the CV method
        depth_CV, CV_test_accuracy = CV_find_best_depth_and_accuracy(
            X_train, y_train, X_test, y_test, max_depth
        )
        
        # Store results
        AM_test_accuracies.append(AM_test_accuracy)
        CV_test_accuracies.append(CV_test_accuracy)

        AM_depths.append(depth_AM)
        CV_depths.append(depth_CV)
        oracle_depths.append(oracle_depth_for_test)
        oracle_test_accuracies.append(oracle_test_accuracy)

        # Check for consistency with oracle depth
        if depth_AM == oracle_depth_for_test:
            AM_consistent_with_oracle += 1
        if depth_CV == oracle_depth_for_test:
            CV_consistent_with_oracle += 1
            
         # Calculate cumulative mean test accuracy for AM and CV methods
        mean_AM_accuracy = np.mean(AM_test_accuracies)
        mean_CV_accuracy = np.mean(CV_test_accuracies)
        mean_oracle_accuracy = np.mean(oracle_test_accuracies)

        # Calculate cumulative mean differences between selected depth and oracle depth
        mean_diff_AM_oracle = np.mean(np.abs(np.array(AM_depths) - np.array(oracle_depths)))
        mean_diff_CV_oracle = np.mean(np.abs(np.array(CV_depths) - np.array(oracle_depths)))

        # Calculate consistency frequencies
        AM_consistency_freq = AM_consistent_with_oracle / (i + 1)
        CV_consistency_freq = CV_consistent_with_oracle / (i + 1)
        
        if i == (N-1):
        # Print mean results at each iteration, each on a new line
            print(f"mean_AM_accuracy: {mean_AM_accuracy:.3f}")
            print(f"mean_CV_accuracy: {mean_CV_accuracy:.3f}")
            print(f"mean_oracle_accuracy: {mean_oracle_accuracy:.3f}")
            print(f"mean_diff_AM_oracle: {mean_diff_AM_oracle:.3f}")
            print(f"mean_diff_CV_oracle: {mean_diff_CV_oracle:.3f}")
            print(f"AM consistency frequency with oracle: {AM_consistency_freq:.3f}")
            print(f"CV consistency frequency with oracle: {CV_consistency_freq:.3f}")
            print("-" * 30)  # Separator for readability between iterations
        
    # Bootstrap to estimate standard deviations of means and frequencies
    bootstrap_means = {
        'mean_AM_accuracy': [],
        'mean_CV_accuracy': [],
        'mean_oracle_accuracy': [],
        'mean_diff_AM_oracle': [],
        'mean_diff_CV_oracle': [],
        'AM_consistency_freq': [],
        'CV_consistency_freq': []
    }
    
    data = np.arange(N)  # Indexes for bootstrapping
    for _ in range(bootstrap_samples):
        # Resample indexes with replacement
        sample_indices = np.random.choice(data, N, replace=True)
        
        # Calculate bootstrap sample means
        bootstrap_means['mean_AM_accuracy'].append(np.mean(np.array(AM_test_accuracies)[sample_indices]))
        bootstrap_means['mean_CV_accuracy'].append(np.mean(np.array(CV_test_accuracies)[sample_indices]))
        bootstrap_means['mean_oracle_accuracy'].append(np.mean(np.array(oracle_test_accuracies)[sample_indices]))
        bootstrap_means['mean_diff_AM_oracle'].append(np.mean(np.abs(np.array(AM_depths)[sample_indices] - np.array(oracle_depths)[sample_indices])))
        bootstrap_means['mean_diff_CV_oracle'].append(np.mean(np.abs(np.array(CV_depths)[sample_indices] - np.array(oracle_depths)[sample_indices])))
        bootstrap_means['AM_consistency_freq'].append(np.mean(np.array(AM_depths)[sample_indices] == np.array(oracle_depths)[sample_indices]))
        bootstrap_means['CV_consistency_freq'].append(np.mean(np.array(CV_depths)[sample_indices] == np.array(oracle_depths)[sample_indices]))
    
    # Calculate standard deviations of bootstrapped means
    std_devs = {key: np.std(values) for key, values in bootstrap_means.items()}
    
    # Print standard deviations at the end
    print("Bootstrap Standard Deviations:")
    for key, value in std_devs.items():
        print(f"{key}: {value:.3f}")
    
    # Return final results and standard deviations
    return {
        'mean_AM_accuracy': mean_AM_accuracy,
        'mean_CV_accuracy': mean_CV_accuracy,
        'mean_oracle_accuracy': mean_oracle_accuracy,
        'mean_diff_AM_oracle': mean_diff_AM_oracle,
        'mean_diff_CV_oracle': mean_diff_CV_oracle,
        'AM_consistency_freq': AM_consistency_freq,
        'CV_consistency_freq': CV_consistency_freq,
        'std_devs': std_devs,
        'AM_test_accuracies': AM_test_accuracies,
        'CV_test_accuracies': CV_test_accuracies,
        'oracle_test_accuracies': oracle_test_accuracies,
        'AM_depths': AM_depths,
        'CV_depths': CV_depths,
        'oracle_depths': oracle_depths
    }

In [26]:
# Define the settings
nu_values = [3, 5, 10, 20]
sigma_values = [0.1, 0.3]
N = 500

# Initialize a list to store the results
all_results = []

# Loop through each combination of nu and sigma
for nu in nu_values:
    for sigma in sigma_values:
        # Print the current setting
        print(f"Running with nu = {nu} and sigma = {sigma}")
        
        # Run the evaluation function and store the results
        results = evaluate_multiple_runs(N=N, nu=nu, sigma=sigma)
        
        # Add the nu and sigma settings to the results dictionary
        results['nu'] = nu
        results['sigma'] = sigma
        
        # Append the results to the list
        all_results.append(results)

# Optionally, save results to a file (e.g., a pickle file or JSON for structured data storage)
import pickle
with open("evaluation_results.pkl", "wb") as f:
    pickle.dump(all_results, f)

print("All evaluations completed and results saved.")

Running with nu = 3 and sigma = 0.1
mean_AM_accuracy: 0.958
mean_CV_accuracy: 0.954
mean_oracle_accuracy: 0.962
mean_diff_AM_oracle: 0.062
mean_diff_CV_oracle: 0.420
AM consistency frequency with oracle: 0.938
CV consistency frequency with oracle: 0.602
------------------------------
Bootstrap Standard Deviations:
mean_AM_accuracy: 0.001
mean_CV_accuracy: 0.002
mean_oracle_accuracy: 0.001
mean_diff_AM_oracle: 0.011
mean_diff_CV_oracle: 0.025
AM_consistency_freq: 0.011
CV_consistency_freq: 0.022
Running with nu = 3 and sigma = 0.3
mean_AM_accuracy: 0.662
mean_CV_accuracy: 0.655
mean_oracle_accuracy: 0.681
mean_diff_AM_oracle: 0.714
mean_diff_CV_oracle: 1.252
AM consistency frequency with oracle: 0.486
CV consistency frequency with oracle: 0.278
------------------------------
Bootstrap Standard Deviations:
mean_AM_accuracy: 0.002
mean_CV_accuracy: 0.002
mean_oracle_accuracy: 0.001
mean_diff_AM_oracle: 0.037
mean_diff_CV_oracle: 0.056
AM_consistency_freq: 0.022
CV_consistency_freq: 0.020


In [1]:
import pickle

# Load the data
with open("evaluation_results.pkl", "rb") as f:
    all_results = pickle.load(f)

In [3]:
import numpy as np
import pandas as pd
# Define unique cases
cases = [(res['nu'], res['sigma']) for res in all_results]
unique_cases = sorted(set(cases))

# Prepare data for the tables
am_regret_data = []
cv_regret_data = []
am_consistency_data = []
cv_consistency_data = []

# Populate tables
for nu, sigma in unique_cases:
    # Filter results for the current case
    results_case = [res for res in all_results if res['nu'] == nu and res['sigma'] == sigma]
    
    # Calculate AM and CV regrets (oracle - method's mean accuracy) with standard deviation
    am_regret_mean = np.mean([res['mean_oracle_accuracy'] - res['mean_AM_accuracy'] for res in results_case])
    am_regret_std = np.mean([res['std_devs'].get('mean_AM_accuracy', 0) for res in results_case])
    cv_regret_mean = np.mean([res['mean_oracle_accuracy'] - res['mean_CV_accuracy'] for res in results_case])
    cv_regret_std = np.mean([res['std_devs'].get('mean_CV_accuracy', 0) for res in results_case])
    
    # Calculate AM and CV consistency frequencies with standard deviation
    am_consistency_mean = np.mean([res['AM_consistency_freq'] for res in results_case])
    am_consistency_std = np.mean([res['std_devs'].get('AM_consistency_freq', 0) for res in results_case])
    cv_consistency_mean = np.mean([res['CV_consistency_freq'] for res in results_case])
    cv_consistency_std = np.mean([res['std_devs'].get('CV_consistency_freq', 0) for res in results_case])
    
    # Append formatted data
    am_regret_data.append(f"{am_regret_mean:.3f} ({am_regret_std:.3f})")
    cv_regret_data.append(f"{cv_regret_mean:.3f} ({cv_regret_std:.3f})")
    am_consistency_data.append(f"{am_consistency_mean:.3f} ({am_consistency_std:.3f})")
    cv_consistency_data.append(f"{cv_consistency_mean:.3f} ({cv_consistency_std:.3f})")

# Convert to DataFrames
columns = [f"nu={nu}, sigma={sigma}" for nu, sigma in unique_cases]
table1 = pd.DataFrame([am_regret_data, cv_regret_data], index=["AM Regret", "CV Regret"], columns=columns)
table2 = pd.DataFrame([am_consistency_data, cv_consistency_data], index=["AM Consistency", "CV Consistency"], columns=columns)



In [5]:
table1

,"nu=3, sigma=0.1","nu=3, sigma=0.3","nu=5, sigma=0.1","nu=5, sigma=0.3","nu=10, sigma=0.1","nu=10, sigma=0.3","nu=20, sigma=0.1","nu=20, sigma=0.3"
AM Regret,0.004 (0.001),0.019 (0.002),0.026 (0.002),0.022 (0.002),0.019 (0.001),0.019 (0.001),0.020 (0.001),0.017 (0.001)
CV Regret,0.007 (0.002),0.026 (0.002),0.033 (0.002),0.030 (0.002),0.026 (0.001),0.023 (0.001),0.022 (0.001),0.019 (0.001)


In [6]:
table2

,"nu=3, sigma=0.1","nu=3, sigma=0.3","nu=5, sigma=0.1","nu=5, sigma=0.3","nu=10, sigma=0.1","nu=10, sigma=0.3","nu=20, sigma=0.1","nu=20, sigma=0.3"
AM Consistency,0.938 (0.011),0.486 (0.022),0.512 (0.022),0.374 (0.021),0.318 (0.021),0.306 (0.021),0.268 (0.020),0.236 (0.019)
CV Consistency,0.602 (0.022),0.278 (0.020),0.412 (0.022),0.270 (0.020),0.200 (0.018),0.228 (0.019),0.180 (0.017),0.172 (0.017)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from collections import Counter

def plot_depth_difference_panel(all_results, output_file="depth_difference_panel.pdf"):
    """
    Plot a panel of bar charts for differences between selected depth and oracle depth for AM and CV
    across different settings of nu and sigma, and save to a PDF file with professional color choices
    and increased font sizes for readability. Adds a reference line at x = 0 for the oracle depth.

    Parameters:
    - all_results: List of results dictionaries containing selected depths and oracle depth lists for each dataset
    - output_file: Filename for the output PDF
    """
    # Define settings for nu and sigma
    nu_values = [3, 5, 10, 20]
    sigma_values = [0.1, 0.3]
    
    # Define professional color choices
    am_color = "#6baed6"  # Light steel blue
    cv_color = "#fcae91"  # Light coral
    
    # Create a PDF file to save the plots
    with PdfPages(output_file) as pdf:
        # Create a 4x2 subplot grid
        fig, axes = plt.subplots(4, 2, figsize=(14, 18))
        fig.suptitle("Distribution of Depth Differences from Oracle Depth", fontsize=20)
        
        # Iterate over settings and plot each in a subplot
        for i, nu in enumerate(nu_values):
            for j, sigma in enumerate(sigma_values):
                # Filter and calculate depth differences
                am_depth_diffs = []
                cv_depth_diffs = []
                
                for res in all_results:
                    if res['nu'] == nu and res['sigma'] == sigma:
                        am_depths = np.array(res['AM_depths'])
                        cv_depths = np.array(res['CV_depths'])
                        oracle_depths = np.array(res['oracle_depths'])
                        
                        am_depth_diffs.extend(am_depths - oracle_depths)
                        cv_depth_diffs.extend(cv_depths - oracle_depths)
                
                # Calculate frequency of each depth difference
                am_counts = Counter(am_depth_diffs)
                cv_counts = Counter(cv_depth_diffs)
                all_diffs = sorted(set(am_counts.keys()).union(cv_counts.keys()))
                am_freqs = [am_counts[diff] for diff in all_diffs]
                cv_freqs = [cv_counts[diff] for diff in all_diffs]
                
                # Plot on the corresponding subplot
                ax = axes[i, j]
                x = np.arange(len(all_diffs))
                width = 0.35
                ax.bar(x - width/2, am_freqs, width, color=am_color, label='AM Depth Difference')
                ax.bar(x + width/2, cv_freqs, width, color=cv_color, label='CV Depth Difference')
                
                # Add a reference line at the position corresponding to zero difference
                if 0 in all_diffs:
                    zero_index = all_diffs.index(0)  # Find the position of zero in all_diffs
                    ax.axvline(x=zero_index, linestyle='--', color='black', linewidth=1)
                
                # Set plot labels and title for each subplot
                ax.set_xlabel('Difference from Oracle Depth', fontsize=12)
                ax.set_ylabel('Frequency', fontsize=12)
                ax.set_title(rf'$\nu = {nu}$, $\sigma = {sigma}$', fontsize=14)
                ax.set_xticks(x)
                ax.set_xticklabels(all_diffs, fontsize=10)
                ax.tick_params(axis='y', labelsize=10)
                ax.grid(axis='y', linestyle='--', alpha=0.7)
        
        # Add a single legend for the entire figure with larger font size
        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right', fontsize=12)
        
        # Adjust layout to prevent overlap
        plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for the suptitle
        
        # Save the figure to the PDF file
        pdf.savefig(fig)
        plt.close(fig)  # Close the figure to free up memory

# Example usage with all_results
# Ensure that `all_results` is loaded or defined with the required data structure.
plot_depth_difference_panel(all_results, output_file="depth_difference_panel.pdf")